## Load Document $\rightarrow$ Split Document $\rightarrow$ Storage $\rightarrow$ Retrieval $\rightarrow$ **Output**

![steps](image.png)

In [2]:
import os
import warnings 
warnings.simplefilter(action="ignore")

import openai

from dotenv import load_dotenv
load_dotenv() 

# openai.api_key  = os.environ['OPENAI_API_KEY']

True

In [3]:
llm_name = "gpt-3.5-turbo"

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

persist_directory = "docs/chroma/"

In [6]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [4]:
print(vectordb._collection.count())

209


In [7]:
question = "What are major topics for this class?"

In [8]:
docs = vectordb.similarity_search(question, k=3)
len(docs)

3

In [9]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [10]:
pretty_print_docs(docs)

Document 1:

statistics for a while or maybe algebra, we'll go over those in the discussion sections as a 
refresher for those of you that want one.  
Later in this quarter, we'll also use the disc ussion sections to go over extensions for the 
material that I'm teaching in the main lectur es. So machine learning is a huge field, and 
there are a few extensions that we really want  to teach but didn't have time in the main 
lectures for.
----------------------------------------------------------------------------------------------------
Document 2:

statistics for a while or maybe algebra, we'll go over those in the discussion sections as a 
refresher for those of you that want one.  
Later in this quarter, we'll also use the disc ussion sections to go over extensions for the 
material that I'm teaching in the main lectur es. So machine learning is a huge field, and 
there are a few extensions that we really want  to teach but didn't have time in the main 
lectures for.
---------------

In [11]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [6]:
# from langchain import HuggingFaceHub

# llm = HuggingFaceHub(
#     repo_id="tiiuae/falcon-7b-instruct",
#     model_kwargs={"temperature": 0.1, "max_length": 512},
# )

### RetrievalQA chain

In [12]:
from langchain.chains import RetrievalQA

In [13]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())

In [14]:
result = qa_chain({"query": question})

In [15]:
result 

{'query': 'What are major topics for this class?',
 'result': 'The major topics for this class are machine learning and its various extensions.'}

In [13]:
result['result']

'\n\nThe major topics for this class include statistics, machine learning, and online resources.'

### Prompt

In [17]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum. Keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [18]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [23]:
question = "Is probability a class topic?"

result = qa_chain({"query": question})

result['result'] 

'Yes, probability is a topic that will be covered in the class. Thanks for asking!'

In [31]:
# pretty_print_docs(result['source_documents'])

In [20]:
question = "What is the current yam price in Mile 12 market?"

result = qa_chain({"query": question})

result['result'] 

"I don't know. Thanks for asking!"

In [22]:
result

{'query': 'What is the current yam price in Mile 12 market?',
 'result': "I don't know. Thanks for asking!",
 'source_documents': [Document(page_content="if this is my house, you may then look at th e straight line and predict that my house is \ngonna go for about that much money, right? Ther e are other decisions that we can make, \nwhich we'll talk about later, which is, well, what if I don' t wanna put a straight line? \nMaybe I should put a quadratic function to it. Ma ybe that fits the data a little bit better. \nYou notice if you do that, the price of my house goes up a bit, so that'd be nice.", metadata={'page': 11, 'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf'}),
  Document(page_content="if this is my house, you may then look at th e straight line and predict that my house is \ngonna go for about that much money, right? Ther e are other decisions that we can make, \nwhich we'll talk about later, which is, well, what if I don' t wanna put a straight line? \nMaybe

### RetrievalQA chain types: `map_reduce` and `refine`

In [34]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_type="mmr"),
    return_source_documents=True,
    chain_type="map_reduce",
)

In [35]:
question = "What are major topics for this class?"
# question = "Is probability a class topic?"

result = qa_chain_mr({"query": question})
print(result)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-OKEaxoW73Lo9pocIXi1wDApB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-OKEaxoW73Lo9pocIXi1wDApB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

{'query': 'What are major topics for this class?', 'result': 'There is no relevant information in the given portion of the document to answer this question.', 'source_documents': [Document(page_content="statistics for a while or maybe algebra, we'll go over those in the discussion sections as a \nrefresher for those of you that want one.  \nLater in this quarter, we'll also use the disc ussion sections to go over extensions for the \nmaterial that I'm teaching in the main lectur es. So machine learning is a huge field, and \nthere are a few extensions that we really want  to teach but didn't have time in the main \nlectures for.", metadata={'page': 8, 'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf'}), Document(page_content="middle of class, but because there won't be video you can safely sit there and make faces \nat me, and that won't show, okay?  \nLet's see. I also handed out this — ther e were two handouts I hope most of you have, \ncourse information handout. So let 

In [36]:
result['result']

'There is no relevant information in the given portion of the document to answer this question.'

In [38]:
qa_chain_refine = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_type="mmr"),
    return_source_documents=True,
    chain_type="refine",
)

In [40]:
question = "What are major topics for this class?"
# question = "Is probability a class topic?"

result = qa_chain_refine({"query": question})
print(result)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection broken: IncompleteRead(1999 bytes read, 4269 more expected)', IncompleteRead(1999 bytes read, 4269 more expected)).


{'query': 'What are major topics for this class?', 'result': 'The major topics covered in this class include:\n\n1. Introduction to Statistics: This topic provides an overview of basic statistical concepts, including data types, measures of central tendency and variability, probability theory, and sampling techniques. Students will learn how to summarize and interpret data using descriptive statistics.\n\n2. Statistical Inference: This topic focuses on hypothesis testing and estimation. Students will learn how to formulate null and alternative hypotheses, conduct hypothesis tests, and make inferences about population parameters based on sample data. They will also explore confidence intervals and the concept of statistical significance.\n\n3. Regression Analysis: This topic delves into linear regression models and their applications. Students will learn how to fit regression models, interpret coefficients, assess model fit, and make predictions. They will also explore multiple regressi